# Log-Loss vs Mean Squared Error

In [1]:
# Imports 
import numpy as np
import pandas as pd

There are many other error functions used for neural networks. Let me teach you another one, called the mean squared error. As the name says, this one is the mean of the squares of the differences between the predictions and the labels.

## Gradient Descent with Squared Errors

Goal: Make predictions as close as possible to the real values.

To measure, we use a metric of how wrong the predictions are using "error." A common metric is the sum of the squared errors.

![a](Images/a1.png)

- where y_hat is the prediction and y is thre true value.
- you take the sum over all output units j and another sum over all data points μ


- This variable j represents the output units of the network. For each output unit, find the difference between the true value yy and the predicted value from the network y_hat, then square the difference, then sum up all those squares.


- μ is a sum over all the data points. So, for each data point you calculate the inner sum of the squared differences for each output unit. Then you sum up those squared differences for each data point. That gives you the overall error for all the output predictions for all the data points.

## Why Squared Errors (SSE)

1. The square ensures the error is always positive, so we only consider magnitude


Remember that the output of a neural network, the prediction, depends on weights.

![a](Images/a2.png)

Our goals is find the weights W_ij that minimize the squared error E. To do this with a neural network we need ***gradient descent***

We must take small steps everytime to minimize the error. We can find this direction by calculating the gradient of the squared error. Gradient is another term for rate of change or slope.

## Gradient 

The gradient is just a derivative generalized to functions with more than one variable. We can use calculus to find the gradient at any point in our error function, which depends on the input weights.

Below I've plotted an example of the error of a neural network with two inputs, and accordingly, two weights. You can read this like a topographical map where points on a contour line have the same error and darker contour lines correspond to larger errors.

At each step, you calculate the error and the gradient, then use those to determine how much to change each weight. Repeating this process will eventually find weights that are close to the minimum of the error function, the black dot in the middle.

![a](Images/a3.png)

### Caveats / Local Mins

Since the weights will just go wherever the gradient takes them, they can end up where the error is low, but not the lowest. These spots are called local minima. If the weights are initialized with the wrong values, gradient descent could lead the weights into a local minimum, illustrated below. There are methods to avoid this, such as using [momentum](https://distill.pub/2017/momentum/).

![a](Images/a4.png)


## The Math Explained

First we need some measure of how bad our prediction are. Just use the difference in output $E = (y - \hat{y})$. We can also add a square to remove negatives and penalize outliers more, so $E = (y - \hat{y})^2$.


Next we need to sum up the error for all data records denoted by the sum over μ. Also multiply 1/2 to clean up math later.

$E = (1/2) \sum_{μ}(y - \hat{y})^2$.

Next sub in the prediction as a function of weights 

![a](Images/a5.png)

So for a single output: 

![a](Images/a6.png)

Next we find the negative for the gradient, which points to the weights with lowest error.

![a](Images/a7.png)

### Update Step

The update step for each weight parameter can be given as following

![a](Images/a8.png)

WE must use chain rule to expand the partial derivative. For

![a](Images/a9.png)

The resulting expansion for the partial derivative of the error with respect weights

![a](Images/a10.png)

$\hat{y}$ depends on the weights so we take the partial derivative of y with respect to the weights.

![a](Images/a11.png)

The partial derivate for $w_{i}$  is just $x_{1}$.

![a](Images/a12.png)

Putting it all together, we get

![a](Images/a15.png)

We can simply define an "ERROR TERM" as  $δ = (y - \hat{y}) * f'(h)$

and write our weight update as:

$w_{i}' = w_{i} + η * δ * x_{i}$

# Gradient Descent: The Code 

From before we saw that one weight update can be calculated as:

$\Delta w_i = \eta \, \delta x_i$ 

with the error term δ as

$\delta = (y - \hat y) f'(h) = (y - \hat y) f'(\sum w_i x_i)$

Remember, in the above equation $(y - \hat y)$ is the output error, and f'(h) refers to the derivative of the activation function, f(h). We'll call that derivative the output gradient.

Now I'll write this out in code for the case of only one output unit. We'll also be using the sigmoid as the activation function f(h).

So f(h) = 1/(1+ exp(-x))

f'(h) = 1/(1+ exp(-x)) * (1 - 1/(1+ exp(-x))

In [7]:
# Functions

# Activation Function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of Activation Function
def sigmoid_prime(x):
    return sigmoid(x) * (1 -sigmoid(x))

In [11]:
learnrate = 0.5
x = np.array([1,2,3,4])
y = np.array(0.5)
w = np.array([0.5, -0.5, 0.3, 0.1])

# Gradient Descent Step:


# Calculate the node's linear combination of inputs and weights
h = np.dot(x,w)
# not h = x * w

# Calculate the ouptut 
y_hat  = sigmoid(h)

# Calculate error of the ouput
error = y - y_hat

# Calculate the error term
error_term = error * sigmoid_prime(h)

# Calculate change in weights
del_w = learnrate * error_term * x

print('Neural Network output:')
print(y_hat)
print('Amount of Error:')
print(error)
print('Change in Weights:')
print(del_w)


Neural Network output:
0.6899744811276125
Amount of Error:
-0.1899744811276125
Change in Weights:
[-0.02031869 -0.04063738 -0.06095608 -0.08127477]


# Implementing gradient descent

Okay, now we know how to update our weights:

$\Delta w_{ij} = \eta * \delta_j * x_i$

You've seen how to implement that for a single update, but how do we translate that code to calculate many weight updates so our network will learn?

As an example, I'm going to have you use gradient descent to train a network on graduate school admissions data (found at http://www.ats.ucla.edu/stat/data/binary.csv). This dataset has three input features: GRE score, GPA, and the rank of the undergraduate school (numbered 1 through 4). Institutions with rank 1 have the highest prestige, those with rank 4 have the lowest.

![a](Images/a16.png)

The goal here is to predict if a student will be admitted to a graduate program based on these features. For this, we'll use a network with one output layer with one unit. We'll use a sigmoid function for the output unit activation.

# Data Cleanup

You might think there will be three input units, but we actually need to transform the data first. The rank feature is categorical, the numbers don't encode any sort of relative values. Rank 2 is not twice as much as rank 1, rank 3 is not 1.5 more than rank 2. Instead, we need to use dummy variables to encode rank, splitting the data into four new columns encoded with ones or zeros. Rows with rank 1 have one in the rank 1 dummy column, and zeros in all other columns. Rows with rank 2 have one in the rank 2 dummy column, and zeros in all other columns. And so on.

We'll also need to standardize the GRE and GPA data, which means to scale the values such that they have zero mean and a standard deviation of 1. This is necessary because the sigmoid function squashes really small and really large inputs. The gradient of really small and large inputs is zero, which means that the gradient descent step will go to zero too. Since the GRE and GPA values are fairly large, we have to be really careful about how we initialize the weights or the gradient descent steps will die off and the network won't train. Instead, if we standardize the data, we can initialize the weights easily and everyone is happy.

Now that the data is ready, we see that there are six input features: gre, gpa, and the four rank dummy variables. Here is an table of the data.

![a](Images/a17.png)

# Mean Square Error

We're going to make a small change to how we calculate the error here. Instead of the SSE, we're going to use the mean of the square errors (MSE). Now that we're using a lot of data, summing up all the weight steps can lead to really large updates that make the gradient descent diverge.

To compensate for this, you'd need to use a quite small learning rate. Instead, we can just divide by the number of records in our data, mm to take the average.This way, no matter how much data we use, our learning rates will typically be in the range of 0.01 to 0.001. Then, we can use the MSE (shown below) to calculate the gradient and the result is the same as before, just averaged instead of summed.

$E = (1/2m) \sum_{\mu} (y^\mu - \hat{y}^\mu)^2 $

Here's the general algorithm for updating the weights with gradient descent:

- Set the weight step to zero: $\Delta w_i = 0$

For each record in the training data
 1. calculate the output $\hat y = f(\sum_i w_i x_i)$
 2. calculate the error term, $δ=(y − \hat{y}) * f'(\sum_{i} w_{i}x_{i})$
 3. update the weight, $Δw' =Δw +δx_{i}$

- Update the weights, $w = w + ηΔw / m$ , where η is the learning rate and mm is the number of records.
- Repeat for set epochs

Our Activation Function remains sigmoid 

$f(h) = 1/(1+e^{-h})$ , and 


$f'(h) = f(h)(1 - f(h))$  so $f'(h) = 1/(1+e^{-h}) * (1 - 1/(1+e^{-h}))$

# Using numpy to implement

1. Initialize the weights
       - Must bbe near 0 and not squashed at high or low ends
       - Must be random, so use normal distribution centered at 0
   
 Use a scale: $1/ \sqrt{n}$ , where n is the number of inputs. This keeps the input low for the number of inputs. So:
 
```bash
weights = np.random.normal(scale=1/n_features**.5, size=n_features)

```

 2. Use the np.dot functions to find dot product of two matrices, which is linear combinination of w and x.
 
```bash
# input to the output layer
output_in = np.dot(weights, inputs)

```
 
3. Split training and testing 90/10

```bash
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)
```

4. Splitting Features and Targets
```bash
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
```

# Implementation

In [16]:
# Read Data
admissions = pd.read_csv('intro_to_neural_network_2.csv')

In [17]:
# Make dummy variables for rank, one hot encoding
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features to zero mean and a standard deviation of 1
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
print(data[:10])

   admit       gre       gpa  rank_1  rank_2  rank_3  rank_4
0      0 -1.798011  0.578348       0       0       1       0
1      1  0.625884  0.736008       0       0       1       0
2      1  1.837832  1.603135       1       0       0       0
3      1  0.452749 -0.525269       0       0       0       1
4      0 -0.586063 -1.208461       0       0       0       1
5      1  1.491561 -1.024525       0       1       0       0
6      1 -0.239793 -1.077078       1       0       0       0
7      0 -1.624876 -0.814312       0       1       0       0
8      1 -0.412928  0.000263       0       0       1       0
9      0  0.972155  1.392922       0       1       0       0


In [18]:
# Split 90% to Training and 10% Testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

print(features[:5])

print ("\n Targets: \n")

print(targets[:5])

          gre       gpa  rank_1  rank_2  rank_3  rank_4
209 -0.066657  0.289305       0       1       0       0
280  0.625884  1.445476       0       1       0       0
33   1.837832  1.603135       0       0       1       0
210  1.318426 -0.131120       0       0       0       1
93  -0.066657 -1.208461       0       1       0       0

 Targets: 

209    0
280    0
33     1
210    0
93     0
Name: admit, dtype: int64


In [21]:
# Helper Functions

def sigmoid(x): # x is dot product of weights and inputs
    return 1/(1 + np.exp(-x))
    

In [22]:
# Gradient Descent

#Initialize the weights
n_records, n_features = features.shape
weights = np.random.normal(scale = 1/n_features**0.5, size = n_features)

#Training Parameters
epochs = 1000
leanrate = 0.5

for e in range(epochs):
    
    # Create a matrix matching shape of weights, this is delta w
    del_w = np.zeros(weights.shape)
    
    